In [1]:
import sys
sys.path.append("..")
from Functions.UNSW_DF import *

from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
#from keras_visualizer import visualizer
#from ann_visualizer.visualize import ann_viz
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
x_train, x_test, y_train, y_test = DF_XY()

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(20, input_dim=53, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
predict_x=model.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)

In [ ]:
ann_accuracy = []
ann_precision = []
ann_recall = []
ann_f1 = []
ann_epochs = []
ann_no_of_layers = []

for epoch in range(1, 100):
    print(f"\n## ---------- EPOCH {i} ----------- ##\n")
    # fit the keras model on the dataset
    model.fit(x_train, y_train, epochs=epoch, batch_size=100)
    
    # predict probabilities for test set
    yhat_probs = model.predict(x_test, verbose=0)
    # predict crisp classes for test set
    yhat_classes = model.predict_classes(x_test, verbose=0)
    
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    ann_accuracy.append(accuracy)
    print('Accuracy: %f' % accuracy)

    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    ann_precision.append(precision)
    print('Precision: %f' % precision)

    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    ann_recall.append(recall)
    print('Recall: %f' % recall)

    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    ann_f1.append(f1)
    print('F1 score: %f' % f1)
    
    ann_no_of_layers.append(len(model.layers))
    ann_epochs.append(epoch)
    

In [ ]:
# evaluate the keras model for training and testing data
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, test_acc = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy for training\t: %.2f' % (train_acc*100))
print('Accuracy for testing\t: %.2f' % (test_acc*100))

# Neural Network

In [ ]:
# dictionary of lists 
dict = {#'epochs': ann_epochs, 
        #'number_of_layers': ann_no_of_layers, 
        'accuracy': ann_accuracy, 
        'precision': ann_precision, 
        'f1': ann_f1, 
        'recall': ann_recall
        } 
    
df = pd.DataFrame(dict, index=ann_epochs)
# EXPORT AS CSV when done.
#df.to_csv('ANN_3_layers.csv')
df

In [ ]:
#UNSW_barplot(data=df, to_range=29 ,x_label="epochs", y_label="Score", title="ANN", x_size=30, y_size=8)

In [ ]:
#ann_viz(model, title="Neural Network")

In [ ]:
#visualizer(model, format='png', view=True)